<a href="https://colab.research.google.com/github/AhmadJamal01/Floodead-Inside/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.utils.data as data
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision import models, transforms
from torch.utils.data import DataLoader

## Read the dataset

In [2]:
!pip install gdal > /dev/null

In [ ]:
import gdown
gdown.download("https://drive.google.com/uc?id=1och-QmNa3FAiS-wssgzCwISbmpSezIi_", "dataset.zip", quiet=False)
gdown.extractall("dataset.zip")

## Prepare the data

In [4]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

path = 'dataset/'
dataset = torchvision.datasets.ImageFolder(root=path, transform=data_transforms)

# Split the dataset into train and test
TRAIN_SIZE = 0.8
train_size = int(TRAIN_SIZE * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = data.random_split(dataset, [train_size, test_size],generator=torch.Generator().manual_seed(42))

# Define the dataloaders
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=True)



## Model Architecture

In [5]:
# Define ResNet model 
model = models.resnet18(pretrained=True)

#  Modify the first layer to be able to handle our data
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the last layer to work with two classes
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# Move the model to GPU
model.cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 347MB/s]


## Training

In [6]:
# Train the model
num_epochs = 15
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 10 == 9:
            print('[Epoch %d, Batch %d] loss: %.3f' % (epoch+1, i+1, running_loss / 10))
            running_loss = 0.0

[Epoch 1, Batch 10] loss: 0.644
[Epoch 1, Batch 20] loss: 0.493
[Epoch 2, Batch 10] loss: 0.382
[Epoch 2, Batch 20] loss: 0.256
[Epoch 3, Batch 10] loss: 0.222
[Epoch 3, Batch 20] loss: 0.171
[Epoch 4, Batch 10] loss: 0.204
[Epoch 4, Batch 20] loss: 0.161
[Epoch 5, Batch 10] loss: 0.091
[Epoch 5, Batch 20] loss: 0.114
[Epoch 6, Batch 10] loss: 0.155
[Epoch 6, Batch 20] loss: 0.129
[Epoch 7, Batch 10] loss: 0.095
[Epoch 7, Batch 20] loss: 0.086
[Epoch 8, Batch 10] loss: 0.067
[Epoch 8, Batch 20] loss: 0.062
[Epoch 9, Batch 10] loss: 0.059
[Epoch 9, Batch 20] loss: 0.150
[Epoch 10, Batch 10] loss: 0.091
[Epoch 10, Batch 20] loss: 0.065
[Epoch 11, Batch 10] loss: 0.109
[Epoch 11, Batch 20] loss: 0.102
[Epoch 12, Batch 10] loss: 0.036
[Epoch 12, Batch 20] loss: 0.097
[Epoch 13, Batch 10] loss: 0.066
[Epoch 13, Batch 20] loss: 0.092
[Epoch 14, Batch 10] loss: 0.082
[Epoch 14, Batch 20] loss: 0.109
[Epoch 15, Batch 10] loss: 0.123
[Epoch 15, Batch 20] loss: 0.083


## Evaluation

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        # Move the inputs and labels to the GPU
        inputs = inputs.cuda()
        labels = labels.cuda()

        # Pass the inputs through the model to obtain the predicted outputs
        outputs = model(inputs)

        # Select the class with the highest probability
        _, predicted = torch.max(outputs.data, 1)

        # Convert the GPU tensors to CPU tensors and then to NumPy arrays to concatenate them with the existing y_true and y_pred lists
        y_true += labels.cpu().numpy().tolist()
        y_pred += predicted.cpu().numpy().tolist()

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print('Accuracy: {:.2f}%'.format(accuracy * 100))
print('F1 Score: {:.2f}'.format(f1))

Accuracy: 93.51%
F1 Score: 0.93


## Saving the model

In [8]:
# save model
torch.save(model.state_dict(), 'resnet18.pth')